# Fitting

There are two built in fitting engines, `lmfit` and `bumps`.

Import all the packages.

In [ ]:
# Import all the packages
import numpy as np
from easyDiffractionLib.sample import Sample
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters

from easyscience.fitting.fitter import Fitter

from easyscience.Datasets.xarray import xr

In [ ]:
try:
    import hvplot.xarray
except:
    import sys
    !{sys.executable} -m pip install hvplot
    import hvplot.xarray

## Preparing the sample and data

We load up a cif file and then set the experimental parameters.

In [ ]:
interface = InterfaceFactory()
c = Phases.from_cif_file('PbSO4.cif')
S = Sample(phases=c, parameters=Instrument1DCWParameters.default(), interface=interface)

We can load up some experimental data

In [ ]:
file_path = 'PbSO4_neutrons.xye'
data_x, data_y, data_e = np.loadtxt(file_path, unpack=True)
data_y = data_y/100.0
data_set = xr.Dataset()
data_set.easyscience.add_coordinate('tth', data_x)
data_set.easyscience.add_variable('I', ['tth'], data_y)
data_set.easyscience.sigma_attach('I', data_e)

In [ ]:
# Generate the simulation y-data and show the xarray
data_set.easyscience.add_variable('simulated', ['tth'], interface.fit_func(data_set['tth']))
data_set

In [ ]:
data_set.hvplot(y=['I', 'simulated'])

We do not have the correct experimental parameters, let's approximate some

In [ ]:
S.parameters.wavelength = 1.54
S.parameters.u_resolution = 0.031
S.parameters.v_resolution = -0.052
S.parameters.w_resolution = 0.032
S.parameters.x_resolution = 0.015
S.parameters.y_resolution = 0.0

In [ ]:
data_set.easyscience.add_variable('simulated', ['tth'], interface.fit_func(data_set['tth']))
data_set.hvplot(y=['I', 'simulated'])

Now we need to set the background

In [ ]:
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint

bg = PointBackground(linked_experiment='PbSO4')
bg.append(BackgroundPoint.from_pars(data_x[0], 1.66))
bg.append(BackgroundPoint.from_pars(16, 0.36))
bg.append(BackgroundPoint.from_pars(50, 1.20))
bg.append(BackgroundPoint.from_pars(100, 1.00))
bg.append(BackgroundPoint.from_pars(data_x[-1], 1.64))
print(bg)

S.set_background(bg)
print(S.backgrounds)

In [ ]:
data_set.easyscience.add_variable('simulated', ['tth'], interface.fit_func(np.array(data_set['tth'])))
data_set.hvplot(y=['I', 'simulated'])

Select CrysFML calculator

In [ ]:
print(interface.available_interfaces)
print(interface.current_interface_name)
interface.switch('CrysFML')
print(interface.current_interface_name)

Reload the sample

In [ ]:
S = Sample(phases=c, parameters=Instrument1DCWParameters.default(), interface=interface)
#S.parameters.wavelength = 1.54
#S.parameters.u_resolution = 0.031
#S.parameters.v_resolution = -0.052
#S.parameters.w_resolution = 0.032
#S.parameters.x_resolution = 0.015
#S.parameters.y_resolution = 0.0
bg = PointBackground(linked_experiment='PbSO4')
bg.append(BackgroundPoint.from_pars(data_x[0], 1.66))
bg.append(BackgroundPoint.from_pars(16, 0.36))
bg.append(BackgroundPoint.from_pars(50, 1.20))
bg.append(BackgroundPoint.from_pars(100, 1.00))
bg.append(BackgroundPoint.from_pars(data_x[-1], 1.64))
print(bg)

S.set_background(bg)

In [ ]:
data_set.easyscience.add_variable('simulated', ['tth'], interface.fit_func(np.array(data_set['tth'])))
data_set.hvplot(y=['I', 'simulated'])

Change scale manually

In [ ]:
S.pattern.scale = 2

data_set.easyscience.add_variable('simulated', ['tth'], interface.fit_func(np.array(data_set['tth'])))
data_set.hvplot(y=['I', 'simulated'])

## Fitting to the data

Initalize the fitting engine and apply a few constraints

In [ ]:
f = Fitter(S, interface.fit_func)

# Vary the scale and the BG points
S.pattern.scale.fixed = False
S.pattern.zero_shift.fixed = False
#S.parameters.resolution_u.fixed = False
#S.parameters.resolution_v.fixed = False
#S.parameters.resolution_w.fixed = False
#S.parameters.resolution_x.fixed = False
#S.backgrounds[0][0].y.fixed = False
#S.backgrounds[0][1].y.fixed = False
#S.backgrounds[0][2].y.fixed = False
#S.backgrounds[0][3].y.fixed = False
#S.backgrounds[0][4].y.fixed = False

Select bumps minimizer

In [ ]:
print("available minimizers:", f.available_engines)
print()
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())
print()
print("switch minimizer")
f.switch_engine('bumps')
f_method = 'lm'
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())

Perform the fit

In [ ]:
result = data_set['I'].easyscience.fit(f, fit_kwargs={'weights':1/data_e, 'method':f_method})

In [ ]:
if result.success:
    print("The fit has been successful: {}".format(result.success))
    print("The gooodness of fit is: {}".format(result.goodness_of_fit))
    
data_set['best_fit'] = result.y_calc

In [ ]:
data_set.hvplot(y =['I', 'best_fit'])

In [ ]:
print(f'Scale: {S.pattern.scale}')
print(f'Scale: {S.pattern.zero_shift}')